<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/E%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84(after_spell_check).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		'# df_P_inc.csv'    K_spellcheck.csv    S_ori_model.pt
 B_spellcheck.csv	 E_spellcheck.csv   labels.zip	        val.txt
 classes.txt		 F_ori_model.pt    'My Drive'
'Colab Notebooks'	 H_ori_model.pt     N_spell_check.csv
'답안 작성용 파일.csv'	 J_spellcheck.csv   Q_ori_model.pt


In [36]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/E_spellcheck.csv')

In [37]:
df['digit_3'].value_counts()

381    704
383    568
370    436
382    371
360    147
390     29
Name: digit_3, dtype: int64

In [38]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [41]:
df1=df[df['digit_3']==381] 
df2=df[df['digit_3']==383] 
df3=df[df['digit_3']==370] 
df4=df[df['digit_3']==382] 
df5=df[df['digit_3']==360] 
df6=df[df['digit_3']==390] 

df10=pd.concat([df1, df2, df3, df4]) #*3
df20=pd.concat([df5, df6]) #*10
df10=df10.reset_index(drop=True)
df20=df20.reset_index(drop=True)

In [43]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df10['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df10['AI_id'][n]), 'p'+str(df10['AI_id'][n]), 'p'+str(df10['AI_id'][n])]
  digit_1_list=[df10['digit_1'][n], df10['digit_1'][n], df10['digit_1'][n]]
  digit_2_list=[df10['digit_2'][n], df10['digit_2'][n], df10['digit_2'][n]]
  digit_3_list=[df10['digit_3'][n], df10['digit_3'][n], df10['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df10_to_insert = pd.DataFrame(data_to_insert)
  df10= df10.append(df10_to_insert, ignore_index=True)
  n=n+1

In [44]:
df10

,AI_id,digit_1,digit_2,digit_3,text
0,id_0006526,E,38,381,고객 요청에 의해 하차장에서 건물 폐기물수집운반
1,id_0009726,E,38,381,사업장에서 음식물을 수거해 폐기 운반하는 작업
2,id_0014342,E,38,381,사업장에서 고객의 요청에 의해 사업장 폐기물 운반 처리
3,id_0017705,E,38,381,사업장에서 고객의 요청으로 지정이 폐기물 수집 운반
4,id_0021029,E,38,381,"의료기관에서 나오는 폐기물(기저귀) 수집, 운반업"
...,...,...,...,...,...
8311,pid_0996511,E,38,382,사업체에서 수집 음식물 중간처리
8312,pid_0996511,E,38,382,사업체에서 수집 운반하여 음식물 쓰레기 중간처리
8313,pid_0999066,E,38,382,"건설사 요청에 의해 덤프차를 가지고 건설, 폐기물 수집운반"
8314,pid_0999066,E,38,382,"건설사 요청에 수집운반 덤프차를 가지고 의해 폐기물 건설,"


In [45]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df20['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=10)
  id_list=['p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]),'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]),'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n]), 'p'+str(df20['AI_id'][n])]
  digit_1_list=[df20['digit_1'][n], df20['digit_1'][n], df20['digit_1'][n], df20['digit_1'][n],df20['digit_1'][n], df20['digit_1'][n], df20['digit_1'][n], df20['digit_1'][n],df20['digit_1'][n], df20['digit_1'][n], df20['digit_1'][n]]
  digit_2_list=[df20['digit_2'][n], df20['digit_2'][n], df20['digit_2'][n], df20['digit_2'][n],df20['digit_2'][n], df20['digit_2'][n], df20['digit_2'][n], df20['digit_2'][n],df20['digit_2'][n], df20['digit_2'][n], df20['digit_2'][n]]
  digit_3_list=[df20['digit_3'][n], df20['digit_3'][n], df20['digit_3'][n], df20['digit_3'][n],df20['digit_3'][n], df20['digit_3'][n], df20['digit_3'][n], df20['digit_3'][n],df20['digit_3'][n], df20['digit_3'][n], df20['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df20_to_insert = pd.DataFrame(data_to_insert)
  df20= df20.append(df20_to_insert, ignore_index=True)
  n=n+1

In [46]:
df20

,AI_id,digit_1,digit_2,digit_3,text
0,id_0009087,E,36,360,원수 취수하여 수자원공사에서 지자체 및 여천공단 배수
1,id_0016054,E,36,360,맑은 물 처리 생산 정수장에서 각 가정에 보급
2,id_0017410,E,36,360,사업장에서 공장 공업용수 공급
3,id_0017994,E,36,360,"공공행정 생활용수 공급, 하수처리 상. 하수도 서비스"
4,id_0048849,E,36,360,"사무실에서 시설, 정수. 생활용수 공급"
...,...,...,...,...,...
2107,pid_0962962,E,39,390,선박을 이용해서 바다에서 해양 환경복원
2108,pid_0962962,E,39,390,선박을 이용해서 바다에서 해양 환경복원
2109,pid_0962962,E,39,390,선박을 이용해서 해양 환경복원 바다에서
2110,pid_0962962,E,39,390,이용해서 바다에서 환경복원


기존 데이터 합치기

In [47]:
df=pd.concat([df10, df20])

In [48]:
df['digit_3'].value_counts()

381    2816
383    2272
360    1764
370    1744
382    1484
390     348
Name: digit_3, dtype: int64

In [49]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0006526,E,38,381,고객 요청에 의해 하차장에서 건물 폐기물수집운반
1,id_0009726,E,38,381,사업장에서 음식물을 수거해 폐기 운반하는 작업
2,id_0014342,E,38,381,사업장에서 고객의 요청에 의해 사업장 폐기물 운반 처리
3,id_0017705,E,38,381,사업장에서 고객의 요청으로 지정이 폐기물 수집 운반
4,id_0021029,E,38,381,"의료기관에서 나오는 폐기물(기저귀) 수집, 운반업"
...,...,...,...,...,...
2107,pid_0962962,E,39,390,선박을 이용해서 바다에서 해양 환경복원
2108,pid_0962962,E,39,390,선박을 이용해서 바다에서 해양 환경복원
2109,pid_0962962,E,39,390,선박을 이용해서 해양 환경복원 바다에서
2110,pid_0962962,E,39,390,이용해서 바다에서 환경복원


In [50]:
df.to_csv('df_E_inc.csv', index=False, encoding='utf-8-sig')